In [1]:
from elliptecstage.elliptecstage import ElloStage, ElloException

In [2]:
import serial
from serial.tools.list_ports import comports

In [3]:
# Sélection automatique
motif_elliptec = 'FT230X Basic UART'
appareil_elliptec = next(filter(lambda x: motif_elliptec in str(x), comports()))

In [12]:
def tester_appareil(com, n: int = 0) -> bool:
    cmd = f'{format(n, "01x")}in\n'
    com.write(bytes(cmd, encoding='utf-8'))
    res = str(com.readline(), encoding='utf-8').strip()
    return bool(res)

def lister_appareils(com, début: int = 0, fin: int = 15) -> list:
    return [n for n in range(début, fin+1) if tester_appareil(com, n)]

In [14]:
com = serial.Serial(appareil_elliptec.device, timeout=5)
addresses = lister_appareils(com, 1, 3)
stages = [ElloStage(com, i) for i in addresses]

In [16]:
for i, stage in enumerate(stages):
    # Exemple tiré de https://github.com/augvislab/elliptecstage
    print(f'# Étage {i}')
    
    # L'initialisation se fait à la création de l'objet.
    #print('1. Initialisation', end='')
    #stage.initialize_motor()
    #print('.')
    
    #print(stage.info)
    
    try:
        print('2. Positionnement: ', end='')
        stage.move_home()
        cmd, dat, add = stage.read_message_blocking_position_response()
        print(f'{cmd=!r}, {dat=!r}, {add=!r}')
    
        print('3. Déplacement: ', end='')
        stage.move_absolute(2.0)
        cmd, z, add = stage.read_message_blocking_position_response()
        print(f'{z=!r}')
    except ElloException as e:
        print()
        print('**', e, '**')
        print()
    
    #_ = input('Continuer?')
    #if _.startswith('n'):
    #    break
    

# Étage 0
2. Positionnement: cmd=<ElloDeviceResponses._DEVGET_POSITION: (b'PO', <bound method ElloDeviceUtility.position of <enum 'ElloDeviceUtility'>>)>, dat=0.0, add=1
3. Déplacement: z=1.99951171875
# Étage 1
2. Positionnement: cmd=<ElloDeviceResponses._DEVGET_POSITION: (b'PO', <bound method ElloDeviceUtility.position of <enum 'ElloDeviceUtility'>>)>, dat=2097151.9990234375, add=2
3. Déplacement: z=2.00048828125
# Étage 2
2. Positionnement: cmd=<ElloDeviceResponses._DEVGET_POSITION: (b'PO', <bound method ElloDeviceUtility.position of <enum 'ElloDeviceUtility'>>)>, dat=2097151.9990234375, add=3
3. Déplacement: z=1.99853515625


In [17]:
com.close()